In [1]:
import os
os.environ['NUMEXPR_MAX_THREADS'] = '32'
os.environ['NUMEXPR_NUM_THREADS'] = '30'
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

In [2]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
import random
import warnings

warnings.simplefilter('ignore')

pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

random.seed(42)
np.random.seed(42)


libgomp: Invalid value for environment variable OMP_NUM_THREADS


In [4]:
main = pd.read_csv('../data/train.csv')

sample = pd.read_csv('../data/sample_submit_naive.csv').drop('predict', axis=1)
sample['target'] = -1
test_ids = pd.read_csv('../data/test_ids.csv')

main = pd.concat([main, sample])

In [7]:
from sklearn.model_selection import train_test_split
predict = np.zeros(12800)
metadatas = []
for i in range(1, 6):
    frames_for_metamodel = []
    embeddings_path = f'../embeddings/coles_{i}.csv'
    main_embs = pd.read_csv(embeddings_path)
    new_embs = pd.read_csv(f'../embeddings/wtte_coles_{i}.csv')
    main_embs = main_embs.merge(new_embs, on='user_id')
    train = main[main.target != -1]
    test = main[main.target == -1]
    train_with_embs = train.merge(main_embs, on='user_id')
    test_with_embs = test.merge(main_embs, on='user_id')
    train, test = train_with_embs.loc[~train_with_embs['user_id'].isin(test_ids['user_id'])], train_with_embs.loc[train_with_embs['user_id'].isin(test_ids['user_id'])]
    strat_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=i)

    X, y = train.drop(['time', 'target'], axis=1), train['target']
    scores = []

    models = []
    for train_index, valid_index in strat_kfold.split(train, train['target']):

        X_train, X_val = X.iloc[train_index], X.iloc[valid_index]
        y_train, y_val = y.iloc[train_index], y.iloc[valid_index]
        X_train1 = X_train.drop(['user_id'], axis=1)
        X_val1 = X_val.drop(['user_id'], axis=1)
        model = CatBoostClassifier(
            iterations = 21,
            depth=1,
            learning_rate=0.025,
            eval_metric='AUC',
            early_stopping_rounds=1000,
            task_type="GPU",
            random_seed=42,
            use_best_model = True,
            l2_leaf_reg=5
        )

        model.fit(Pool(X_train1, y_train),
                  eval_set=Pool(X_val1, y_val),
                  verbose=1000)
        models.append(model)

        pred = model.predict_proba(X_val1)[:, 1]
        scores.append(metrics.roc_auc_score(y_val, pred))
        frames_for_metamodel.append(pd.DataFrame({'user_id': X_val.user_id.values, 'pred_emb': pred}))
    metadata = pd.concat(frames_for_metamodel, axis=0).reset_index()
    metadatas.append(metadata)
    for j in range(len(models)):
        predict += models[j].predict_proba(test.drop(['target'], axis=1))[:, 1]

    print(metrics.roc_auc_score(test['target'], predict))

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.5795046	best: 0.5795046 (0)	total: 6.08ms	remaining: 122ms
20:	test: 0.6217932	best: 0.6217932 (20)	total: 106ms	remaining: 0us
bestTest = 0.6217931509
bestIteration = 20


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.5714928	best: 0.5714928 (0)	total: 5.58ms	remaining: 112ms
20:	test: 0.6104253	best: 0.6104253 (20)	total: 91.7ms	remaining: 0us
bestTest = 0.6104252636
bestIteration = 20


Default metric period is 5 because AUC is/are not implemented for GPU


...

0:	test: 0.5729147	best: 0.5729147 (0)	total: 5.54ms	remaining: 111ms
20:	test: 0.6240211	best: 0.6240211 (20)	total: 94.9ms	remaining: 0us
bestTest = 0.6240210831
bestIteration = 20


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.5613149	best: 0.5613149 (0)	total: 4.6ms	remaining: 92ms
20:	test: 0.6255453	best: 0.6255453 (20)	total: 77.6ms	remaining: 0us
bestTest = 0.6255453229
bestIteration = 20


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.5693931	best: 0.5693931 (0)	total: 5.87ms	remaining: 117ms
20:	test: 0.6191851	best: 0.6195283 (9)	total: 99.9ms	remaining: 0us
bestTest = 0.6195282936
bestIteration = 9
Shrink model to first 10 iterations.
0.661730342141184


In [8]:
agg_metadata = metadatas[0].drop('index', axis = 1)
for i in range(1,5):
    agg_metadata=agg_metadata.merge(metadatas[i].drop('index', axis = 1).rename(columns={"pred_emb": f"pred_emb{i}"}), on='user_id')
agg_metadata['pred_emb_sum'] = (agg_metadata['pred_emb'] + agg_metadata['pred_emb1'] + agg_metadata['pred_emb2'] + agg_metadata['pred_emb3'] + agg_metadata['pred_emb4'])/5

In [11]:
test_preds = {'pred_emb_sum': predict/25}
test_preds['user_id'] = test['user_id']
test_preds = pd.DataFrame(test_preds)

In [13]:
final_prediction = pd.concat([agg_metadata[['user_id', 'pred_emb_sum']], test_preds])

In [15]:
from sklearn.model_selection import train_test_split
predict = np.zeros(32000)
for i in range(1, 6):
    frames_for_metamodel = []
    embeddings_path = f'../embeddings/coles_{i}.csv'
    main_embs = pd.read_csv(embeddings_path)
    new_embs = pd.read_csv(f'../embeddings/wtte_coles_{i}.csv')
    main_embs = main_embs.merge(new_embs, on='user_id')
    train = main[main.target != -1]
    test = main[main.target == -1]
    train_with_embs = train.merge(main_embs, on='user_id')
    test_with_embs = test.merge(main_embs, on='user_id')
    strat_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=i)

    X, y = train_with_embs.drop(['time', 'target'], axis=1), train_with_embs['target']
    scores = []

    models = []
    for train_index, valid_index in strat_kfold.split(train_with_embs, train_with_embs['target']):

        X_train, X_val = X.iloc[train_index], X.iloc[valid_index]
        y_train, y_val = y.iloc[train_index], y.iloc[valid_index]
        X_train1 = X_train.drop(['user_id'], axis=1)
        X_val1 = X_val.drop(['user_id'], axis=1)
        model = CatBoostClassifier(
            iterations = 21,
            depth=1,
            learning_rate=0.025,
            eval_metric='AUC',
            early_stopping_rounds=1000,
            task_type="GPU",
            random_seed=42,
            use_best_model = True,
            l2_leaf_reg=5
        )

        model.fit(Pool(X_train1, y_train),
                  eval_set=Pool(X_val1, y_val),
                  verbose=100)
        models.append(model)

        pred = model.predict_proba(X_val1)[:, 1]
        scores.append(metrics.roc_auc_score(y_val, pred))
    for j in range(len(models)):
        predict += models[j].predict_proba(test_with_embs.drop(['target', 'user_id', 'time'], axis=1))[:, 1]


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.5766022	best: 0.5766022 (0)	total: 6.07ms	remaining: 121ms
20:	test: 0.6272591	best: 0.6272591 (20)	total: 111ms	remaining: 0us
bestTest = 0.6272591352
bestIteration = 20


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.5625888	best: 0.5625888 (0)	total: 5.98ms	remaining: 120ms
20:	test: 0.6010063	best: 0.6010063 (20)	total: 101ms	remaining: 0us
bestTest = 0.6010062993
bestIteration = 20


Default metric period is 5 because AUC is/are not implemented for GPU


...

0:	test: 0.5576193	best: 0.5576193 (0)	total: 4.24ms	remaining: 84.9ms
20:	test: 0.6213041	best: 0.6219133 (19)	total: 79.5ms	remaining: 0us
bestTest = 0.6219133139
bestIteration = 19
Shrink model to first 20 iterations.


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.5872072	best: 0.5872072 (0)	total: 5.92ms	remaining: 118ms
20:	test: 0.6524037	best: 0.6525732 (19)	total: 107ms	remaining: 0us
bestTest = 0.6525731981
bestIteration = 19
Shrink model to first 20 iterations.


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.5749865	best: 0.5749865 (0)	total: 6.33ms	remaining: 127ms
20:	test: 0.6283545	best: 0.6283545 (20)	total: 106ms	remaining: 0us
bestTest = 0.6283545494
bestIteration = 20


In [16]:
sbmt_preds = {'pred_emb_sum': predict/25}
sbmt_preds['user_id'] = test['user_id']
sbmt_preds = pd.DataFrame(sbmt_preds)

In [17]:
final_prediction = pd.concat([final_prediction, sbmt_preds])

In [19]:
final_prediction.to_csv("../predicts/coles_coles-wtte-preds.csv", index=False)